<a href="https://colab.research.google.com/github/ju-vargas/Classificacao-e-Pesquisa-de-Dados/blob/main/Lab03/labCPD_03_CPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disciplina de Classificação e Pesquisa de Dados
 
# Laboratório #3 - Final
 
### Implementação e comparação (em C++) dos principais algoritmos de classificação por seleção, intercalação e lineares.

## Exercício I: algoritmos de seleção (Heapsort e relacionados)

Utilizando a versão padrão do `heapsort` que você implementou na aula sobre "**Classificação por seleção: parte 2**", faça uma análise de desempenho dele para arrays de tamanho 100, 1000 e 10000, tanto crescentes quanto decrescentes. Registre o tempo de processamento, a quantidade de trocas e a quantidade de comparaçoes.

Se você não elaborou o algoritmo naquela ocasião, implemente-o agora. Não se esqueça de que ele é um algoritmo local e que usa funções auxiliares (as quais estão disponíveis no material daquela aula). Você pode usar suas próprias funções ou estruturá-lo de outra forma, se desejar, mas ele deve seguir as mesmas regras e preceitos.

Observação: naquela aula não foi solicitado o encaminhamento dos exercícios. Haviam outras tarefas e comparações. Se você desejar um feedback sobre elas, encaminhe suas soluções ao final deste arquivo.

In [17]:
%%writefile hs_test.cpp
#include<iostream>
#include<tuple>
#include<string>
#include<vector>
#include<unordered_map>
#include<bits/stdc++.h>
#include<map>
#include<chrono>

using namespace std;

// Definições e variáveis globais necessárias
typedef int element_t;                                  // tipo do elemento
typedef vector<element_t> array_t;                      // tipo do array
typedef tuple<int, int, std::chrono::duration<double>> loginfo_t;              // armazena informações de desempenho <trocas, comparações, tempo em ms>

void selectionsort(array_t&, loginfo_t&);
void heapsort(array_t&, loginfo_t&);
void buildheap(array_t&, loginfo_t&);
int filho_e(const array_t&, element_t);
int filho_d(const array_t&, element_t);
int pai(const array_t&, element_t);
void heapify(array_t&, element_t, int, loginfo_t&);
int heap_max(const array_t&, loginfo_t&);
int extract_max(array_t&, loginfo_t&);
void heap_insert(array_t&, element_t, loginfo_t&);
void swap(element_t* n1, element_t* n2);

int main(void){

    loginfo_t loginfo;
    int qtd;

    

    array_t array;                  
    array_t array1;                  

//<trocas, comparações, tempo em ms>
//---------------------------------------------------------------------------------------------------------------------
    
    auto start = std::chrono::steady_clock::now();
    auto finish = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds;
    
    cout << "DECRESCENTE" << endl; 

    //array crescente de 10000 elementos
    cout << "10000 elementos" << endl; 
    for(auto i = 10000; i>0; i--){
        array.push_back(i); 
    }

    start = std::chrono::steady_clock::now();
    
    buildheap(array, loginfo);
    heapsort(array, loginfo);
    
    finish = std::chrono::steady_clock::now();
    elapsed_seconds = finish - start;
    get<2>(loginfo) = elapsed_seconds; 

    cout << "trocas = " << get<0>(loginfo) << "; "; 
    cout << "comparacoes = " << get<1>(loginfo) << "; "; 
    cout << "tempo = " << get<2>(loginfo).count() << "ms; " << endl; 
    
    return 0;
}


// =============================================================================================================================
// Algoritmos de ordenação por Seleção

// HEAPSORT 
void heapsort(array_t& array, loginfo_t& loginfo){
    int trocas = 0;
    int comparacoes = 0;
    int heap_size = array.size();               // heap size retorna o  numero de elementos  do heap 
    int qtd_elementos = heap_size-1;            // qnt_elementos é o    numero de elementos  menos 1

    buildheap(array, loginfo);                  // deixa em formato de heap


    for(auto i = qtd_elementos; i>0; i--){      // vindo do último elemento, eu subtraio e percorro enquanto i nao é 0   
        trocas++;                               // se sao 3 elementos, qnt = 2, itero 2 vezes, NUNCA chego no elemento array[0] primeiro
       
        swap(array[i], array[0]);               // troco o ultimo elemento do array com o primeiro do array[0]  (o primeiro é p ser o maior)
        heap_size--;                            // diminuo o tamanho do array (era 3, 2)
        heapify(array, 0, heap_size, loginfo);  // mando o array, o tamanho organizo. ele nao pega o ultimo elemento, q eu coloquei fora
                                                // ele pega o elemento [0] q eu tinha trocado e organiza o heap p esse ter o maior de novo      

    }
    get<0>(loginfo)=trocas;
    get<1>(loginfo)=comparacoes;
}

// HEAPSORT - Funcoes auxiliares
void buildheap(array_t& array, loginfo_t& loginfo){
    int ultimo_pai = (array.size()/2)-1;
    for(auto i=ultimo_pai; i>=0; i--)
        heapify(array, i, array.size(), loginfo);
}
  
int filho_e(const array_t& array, element_t elemento){
    return elemento*2+1;
}
 
int filho_d(const array_t& array, element_t elemento){
    return elemento*2+2;
}
 
int pai(const array_t& array, element_t elemento){
    return (elemento/2);
}
 
// heapify: verifica se o elemento na posição passada é um heap e se não for transforma-o em um
// parâmetros: array, índice do elemento a heapificar, tamanho do heap, dicionário de logs
// ou seja, coloca no lugar
void heapify(array_t& array, element_t elemento, int heap_size, loginfo_t& loginfo){
    // índice do elemento a ser reorganizado: recebe o 0 para o heapsort (pois é o que to mudando sempre)
    //elemente_t eh do tipo int

    element_t esq, dir, maior; 

    esq = filho_e(array, elemento);  
    dir = filho_d(array, elemento);
    maior = elemento;                   //se estivesse correto o índice do maior seria o 0

    //aqui descubro qual filho é maior que o pai (ou se nenhum eh)
    if ((esq < heap_size) & (array[esq] > array[maior])) //se esq eh valido e o valor e maior q o pai 
        maior = esq;
    if ((dir < heap_size) & (array[dir] > array[maior]))
        maior = dir; 

    if (maior != elemento) {
        swap(array[elemento], array[maior]);
        heapify(array, maior, heap_size, loginfo); //heapsize nao muda dentro da funcao, posso mandar o mesmo
    }
}


 
//Máximo(S): Retorna o elemento de S com maior valor de chave
int heap_max(array_t& heap, loginfo_t& loginfo){
    return heap[0];
}

int extract_max(array_t& heap, loginfo_t& loginfo){    
    int heap_size = heap.size(); 
    element_t max; 

    if(heap_size < 1)
        cout << "erro: heap underflow!";  
    max = heap[0]; 
    heap[0] = heap[heap_size]; 
    heap_size = heap_size-1;                    // assim ele tirou o 0 trocando com o ultimo 
    heapify (heap, 0, heap_size, loginfo);      // preciso arrumar o que eu troquei  
    return max;                                 // retorno o valor máximo
}

void heap_insert(array_t& heap, element_t elemento, loginfo_t& loginfo){
    int heap_size; 
    heap_size = heap.size() + 1;                // ja coloco o tamanho aumentado
    
    auto i = heap_size;

    while ((i > 0) && heap[pai(heap, i)] < elemento) {
        heap[i] = heap[pai(heap,i)];
        i = pai(heap,i); 
    }
    heap[i] = elemento; 
}

//OUTRAS funcoes auxiliares
//swap
void swap(element_t* n1, element_t* n2){
	int troca = *n1;
	*n1 = *n2;
	*n2 = troca;
}

Overwriting hs_test.cpp


In [18]:
%%script bash
g++ hs_test.cpp -o hs_test

In [19]:
%%script bash
./hs_test

DECRESCENTE
10000 elementos
trocas = 9999; comparacoes = 0; tempo = 0.00806935ms; 


  TESTE HEAPSORT

    CRESCENTE
    100 elementos
    trocas = 99; comparacoes = 0; tempo = 7.18e-05ms; 

    CRESCENTE
    1000 elementos
    trocas = 999; comparacoes = 0; tempo = 0.00066443ms;

    CRESCENTE
    10000 elementos
    trocas = 9999; comparacoes = 0; tempo = 0.00579099ms;  

    DECRESCENTE
    100 elementos
    trocas = 99; comparacoes = 0; tempo = 3.7698e-05ms; 

    DECRESCENTE
    1000 elementos
    trocas = 999; comparacoes = 0; tempo = 0.000568995ms; 

    DECRESCENTE
    10000 elementos
    trocas = 9999; comparacoes = 0; tempo = 0.00806935ms; 


## Exercício II: algoritmos de intercalação

A seguir você encontra uma **versão do algoritmo de merge** e um exemplo de sua aplicação.

Depois, há vários **algoritmos a implementar**, a saber:
1. `two_way_merge`. 
2. `multi_way_merge`.
2. `merge_sort`.

Implemente-os conforme o comportamento esperado deles. Depois, teste-os com arrays criados por você. Não se esqueça que `two_way_merge` e `multi_way_merge` trabalham em cima de arrays previamente ordenados. Já o `merge_sort` aceita qualquer tipo de array. 

In [26]:
%%writefile sorting4.cpp
#include<iostream>
#include<tuple>
#include<string>
#include<vector>
#include<unordered_map>
#include<bits/stdc++.h>
#include<map>

using namespace std;

// Definições e variáveis globais necessárias
typedef int element_t;                                                          // tipo do elemento
typedef vector<element_t> array_t;                                              // tipo do array
typedef tuple<int, int, double> loginfo_t;                                      // armazena informações de desempenho <trocas, comparações, tempo em ms>

void merge(const array_t&, const array_t&, array_t&, loginfo_t&);
void two_way_merge(const vector<array_t>, array_t&, loginfo_t&);
void multi_way_merge(const vector<array_t>, array_t&, loginfo_t&);
void mergesort(array_t&, loginfo_t&);
void swap(element_t* n1, element_t* n2);
void merge_aux(array_t&, int, int, int, loginfo_t&);
void mergesort_aux(array_t&, int, int, loginfo_t&);

int main(void){
   loginfo_t loginfo;
   int qtd;
   array_t array1 = {1,2,4,6,7,8,15,16,22,23,24,30};
   array_t array2 = {1,2,3,4,5,6,7,8,25,27,35,44,65};
   array_t array3, array4;
   array_t array5 = {1,9,8,72,64,1,1,32,1,52,2};
   array_t array6 = {1,1,1,1,8,9,32,64,72};

   cout << endl << "Array 1: " << endl ; 
   for(auto e:array1) cout << e << " ";  
   cout << endl;

   cout << endl << "Array 2: " << endl; 
   for(auto e:array2) cout << e << " ";  
   cout << endl;

   merge(array1, array2, array3, loginfo);
   
   cout << endl << "Merge: " << endl; 
   for(auto e:array3) cout << e << " ";  
   cout << endl;
   cout << "trocas = " << get<0>(loginfo) << "; "; 
   cout << "comparacoes = " << get<1>(loginfo) << "; "; 
   get<0>(loginfo) = 0; 
   get<1>(loginfo) = 0; 

   vector<array_t> arrays = { array1, array2, array3};
   two_way_merge(arrays, array4, loginfo);
   

   cout << endl << endl << "Two Way Merge: " << endl; 
   for(auto e:array4) cout << e << " ";  
   cout << endl;
   cout << "trocas = " << get<0>(loginfo) << "; "; 
   cout << "comparacoes = " << get<1>(loginfo) << "; "; 
   get<0>(loginfo) = 0; 
   get<1>(loginfo) = 0; 

   array4.clear(); 

   multi_way_merge(arrays, array4, loginfo);

   cout << endl << endl <<  "Multi Way Merge: " << endl; 
   for(auto e:array4) cout << e << " ";  
   cout << endl;
   cout << "trocas = " << get<0>(loginfo) << "; "; 
   cout << "comparacoes = " << get<1>(loginfo) << "; "; 
   get<0>(loginfo) = 0; 
   get<1>(loginfo) = 0; 

   cout << endl << endl << "Array 5: " << endl; 
   for(auto e:array5) cout << e << " ";  

   mergesort(array5, loginfo);
   

   cout << endl << endl << "Merge Sort: " << endl; 
   for(auto e:array5) cout << e << " ";  
   cout << endl;
   cout << "trocas = " << get<0>(loginfo) << "; "; 
   cout << "comparacoes = " << get<1>(loginfo) << "; "; 
   get<0>(loginfo) = 0; 
   get<1>(loginfo) = 0; 

   // TODO: mostrar log de operações

   return 0;
}  

// ======================================================================================================================================
// Algoritmos de intercalação
void merge(const array_t& array1, const array_t& array2, array_t& array_final, loginfo_t& loginfo){
   int i = 0, j = 0, trocas = 0, comparacoes = 0;
   int qtd_a1 = array1.size();
   int qtd_a2 = array2.size();
    
   bool elementos = true;
   array_final.clear();    
   while(i<qtd_a1 && j<qtd_a2){
      if(array1[i] <= array2[j]) 
         array_final.push_back(array1[i++]);          
      else
         array_final.push_back(array2[j++]);  
      comparacoes++;      
      trocas++;
    }
    
   if(j<qtd_a2 && i>=qtd_a1)                                                   // array 1 terminou
      for(auto e=j;e<qtd_a2;e++) {
         array_final.push_back(array2[e]);
         trocas++;
      }
    
   if(i<qtd_a1 && j>=qtd_a2)                                                   // array 2 terminou
      for(auto e=i;e<qtd_a1;e++) {
         array_final.push_back(array1[e]);
         trocas++;
      }

   get<0>(loginfo)=trocas;
   get<1>(loginfo)=comparacoes;
    // TODO: atualizar loginfo
}

// Recebe uma lista de arrays e intercala-os 2 a 2
// retorna um array com o resultado da intercalação
void two_way_merge(const vector<array_t> arrays, array_t& array_final, loginfo_t& loginfo){   
   // TODO: atualizar loginfo
   int n = arrays.size(); 

   //arrays vazio:
   if (n == 0) {
      return; 
   }     

   // começo colocando o primeiro array em auxiliar
   std::vector<int> auxiliar = arrays[0];
   std::vector<int> combinado;
 
   for (int i = 1; i < n; i++ ) {   
      //agora intercala ordenando      
      merge(auxiliar, arrays[i], combinado, loginfo);

      //substitui auxiliar pelo resultado
      auxiliar = combinado; 
      combinado.clear();  
   }

   //coloco o resultado em array_final
   array_final = auxiliar; 
}

void mergesort (array_t& array, loginfo_t& loginfo) {
   // TODO: atualizar loginfo
 
   int n = array.size();
    
   if (n < 2) {
        return;
    }
   int mid = n / 2;
   vector<int> left(array.begin(), array.begin() + mid);
   vector<int> right(array.begin() + mid, array.end());
   mergesort(left, loginfo);
   mergesort(right, loginfo);
   merge(left, right, array, loginfo);
}

// Recebe uma lista de arrays e intercala-os usando estrutura similar a heap-min
// fiz diferente!!
void multi_way_merge(const vector<array_t> arrays, array_t& array_final, loginfo_t& loginfo){           
   // TODO: implementar    
   // TODO: atualizar loginfo
   int trocas = 0; 
   int comparacoes = 0; 
  
  int n = arrays.size();
    vector<int> idx(n, 0); // vetor de índices 

    while (true) {
        int min_element = INT_MAX;
        int min_array_idx = -1;

        // encontra o menor elemento entre os elementos atuais de cada array
        for (int i = 0; i < n; i++) {
            // vejo se já nao acabou o array, e se eh menor q o min elemento. se eh, atualizo
            // o min_array_idx serve pra saber se ja acabou, pq dai nao entra aqui
            comparacoes++;
            if (idx[i] < arrays[i].size() && arrays[i][idx[i]] < min_element) { 
                min_element = arrays[i][idx[i]];
                min_array_idx = i;
            }
        }

        if (min_array_idx == -1) {
            break; // todos os elementos dos arrays já foram processados
        }

        // adiciona o menor elemento encontrado ao resultado final
        array_final.push_back(min_element);
        trocas++; 

        // atualiza o índice correspondente ao array do elemento adicionado
        // ou seja, sobe
        idx[min_array_idx]++;
    }
    get<0>(loginfo)=trocas;
    get<1>(loginfo)=comparacoes;
}




Overwriting sorting4.cpp


In [27]:
%%script bash
g++ sorting4.cpp -o sorting4

In [28]:
%%script bash
./sorting4


Array 1: 
1 2 4 6 7 8 15 16 22 23 24 30 

Array 2: 
1 2 3 4 5 6 7 8 25 27 35 44 65 

Merge: 
1 1 2 2 3 4 4 5 6 6 7 7 8 8 15 16 22 23 24 25 27 30 35 44 65 
trocas = 25; comparacoes = 22; 

Two Way Merge: 
1 1 1 1 2 2 2 2 3 3 4 4 4 4 5 5 6 6 6 6 7 7 7 7 8 8 8 8 15 15 16 16 22 22 23 23 24 24 25 25 27 27 30 30 35 35 44 44 65 65 
trocas = 50; comparacoes = 49; 

Multi Way Merge: 
1 1 1 1 2 2 2 2 3 3 4 4 4 4 5 5 6 6 6 6 7 7 7 7 8 8 8 8 15 15 16 16 22 22 23 23 24 24 25 25 27 27 30 30 35 35 44 44 65 65 
trocas = 50; comparacoes = 153; 

Array 5: 
1 9 8 72 64 1 1 32 1 52 2 

Merge Sort: 
1 1 1 1 2 8 9 32 52 64 72 
trocas = 11; comparacoes = 9; 

CONTANDO PUSH NO VECTOR COMO TROCAS!

    
    Array 1: 
    1 2 4 6 7 8 15 16 22 23 24 30 

    Array 2: 
    1 2 3 4 5 6 7 8 25 27 35 44 65 

    Merge: 
    1 1 2 2 3 4 4 5 6 6 7 7 8 8 15 16 22 23 24 25 27 30 35 44 65 
    trocas = 25; comparacoes = 22; 

    Two Way Merge: 
    1 1 1 1 2 2 2 2 3 3 4 4 4 4 5 5 6 6 6 6 7 7 7 7 8 8 8 8 15 15 16 16 22 22 23 23 24 24 25 25 27 27 30 30 35 35 44 44 65 65 
    trocas = 50; comparacoes = 49; 

    Multi Way Merge: 
    1 1 1 1 2 2 2 2 3 3 4 4 4 4 5 5 6 6 6 6 7 7 7 7 8 8 8 8 15 15 16 16 22 22 23 23 24 24 25 25 27 27 30 30 35 35 44 44 65 65 
    trocas = 50; comparacoes = 153; 

    Array 5: 
    1 9 8 72 64 1 1 32 1 52 2 

    Merge Sort: 
    1 1 1 1 2 8 9 32 52 64 72 
    trocas = 11; comparacoes = 9; 

## Exercício 3: Avaliação do desempenho do `merge_sort`

Utilizando o `merge_sort` que você implementou acima, faça uma análise de desempenho dele para arrays de tamanho 100, 1000 e 10000, tanto crescentes quanto decrescentes. Registre o tempo de processamento, a quantidade de trocas e a quantidade de comparações.


In [55]:
%%writefile ms_test.cpp
#include<iostream>
#include<tuple>
#include<string>
#include<vector>
#include<unordered_map>
#include<bits/stdc++.h>
#include<map>

using namespace std;

// Definições e variáveis globais necessárias
typedef int element_t;                                                          // tipo do elemento
typedef vector<element_t> array_t;                                              // tipo do array
typedef tuple<int, int, std::chrono::duration<double>> loginfo_t;                                      // armazena informações de desempenho <trocas, comparações, tempo em ms>

void merge(const array_t&, const array_t&, array_t&, loginfo_t&);
void mergesort(array_t&, loginfo_t&);
void swap(element_t* n1, element_t* n2);


int main(void){
    loginfo_t loginfo;
    int qtd;
    array_t array1 = {1,2,4,6,7,8,15,22,23};
    array_t array2 = {1,2,3,4,5,6,7,8,25,27,35,44};
    array_t array3, array4;
    array_t array5 = {1,9,8,72,64,1,1,32,1};
    array_t array6 = {1,1,1,1,8,9,32,64,72};

    array_t array; 

    auto start = std::chrono::steady_clock::now();
    auto finish = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds;
    

   cout << "Merge Sort: " << endl << endl; 
   cout << "DECRESCENTE" << endl; 

   //array decrescente de 10000 elementos
   cout << "10000 elementos" << endl; 
   for(auto i = 10000; i>0; i--){
      array.push_back(i); 
   }                 

   
    start = std::chrono::steady_clock::now();
    mergesort(array, loginfo);
    finish = std::chrono::steady_clock::now();
    elapsed_seconds = finish - start;
    get<2>(loginfo) = elapsed_seconds; 
   
    cout << "trocas = " << get<0>(loginfo) << "; "; 
    cout << "comparacoes = " << get<1>(loginfo) << "; "; 
    cout << "tempo = " << get<2>(loginfo).count() << "ms; " << endl; 

   get<0>(loginfo) = 0; 
   get<1>(loginfo) = 0; 
   

   return 0;
}  

// ======================================================================================================================================
// Algoritmos de intercalação
void merge(const array_t& array1, const array_t& array2, array_t& array_final, loginfo_t& loginfo){
   int i = 0, j = 0, trocas = 0, comparacoes = 0;
   int qtd_a1 = array1.size();
   int qtd_a2 = array2.size();
    
   bool elementos = true;
   array_final.clear();    
   while(i<qtd_a1 && j<qtd_a2){
      if(array1[i] <= array2[j]) 
         array_final.push_back(array1[i++]);          
      else
         array_final.push_back(array2[j++]);  
      comparacoes++;      
      trocas++;
    }
    
   if(j<qtd_a2 && i>=qtd_a1)                                                   // array 1 terminou
      for(auto e=j;e<qtd_a2;e++) {
         array_final.push_back(array2[e]);
         trocas++;
      }
    
   if(i<qtd_a1 && j>=qtd_a2)                                                   // array 2 terminou
      for(auto e=i;e<qtd_a1;e++) {
         array_final.push_back(array1[e]);
         trocas++;
      }

   get<0>(loginfo)=trocas;
   get<1>(loginfo)=comparacoes;
    // TODO: atualizar loginfo
}

void mergesort (array_t& array, loginfo_t& loginfo) {
   // TODO: atualizar loginfo
 
   int n = array.size();
    
   if (n < 2) {
        return;
    }
   int mid = n / 2;
   vector<int> left(array.begin(), array.begin() + mid);
   vector<int> right(array.begin() + mid, array.end());
   mergesort(left, loginfo);
   mergesort(right, loginfo);
   merge(left, right, array, loginfo);
}






Overwriting ms_test.cpp


In [56]:
%%script bash
g++ ms_test.cpp -o ms_test
./ms_test

Merge Sort: 

DECRESCENTE
10000 elementos
trocas = 10000; comparacoes = 5000; tempo = 0.00853869ms; 


MERGE SORT 
(fiquei com dúvida em como contar trocas e comparações)
    
    
    CRESCENTE
    100 elementos
    trocas = 100; comparacoes = 50; tempo = 6.8195e-05ms;

    CRESCENTE
    1000 elementos
    trocas = 1000; comparacoes = 500; tempo = 0.000769853ms; 

    CRESCENTE
    10000 elementos
    trocas = 10000; comparacoes = 5000; tempo = 0.00840308ms;  

    DECRESCENTE
    100 elementos
    trocas = 100; comparacoes = 50; tempo = 8.2454e-05ms

    DECRESCENTE
    1000 elementos
    trocas = 1000; comparacoes = 500; tempo = 0.00077753ms;

    DECRESCENTE
    10000 elementos
    trocas = 10000; comparacoes = 5000; tempo = 0.00853869ms; 

## Parte IV: algoritmos de tempo linear

Implemente os algoritmos:

1. `RADIX-sort LSD` 
2. `RADIX-sort MSD` 
3. `CountingSort`

Depois, teste o seu desempenho (tempo de processmento) para ordenar um array decrescente de 100 elementos.

Counting Sort: 

In [ ]:
%%writefile ms_test.cpp

#include <iostream>
#include <vector>
#include<bits/stdc++.h>
#include<chrono>

using namespace std;

// Definições e variáveis globais necessárias
typedef int element_t;                                                      // tipo do elemento
typedef vector<element_t> array_t;                                          // tipo do array
typedef tuple<int, int, std::chrono::duration<double>> loginfo_t;           // armazena informações de desempenho <trocas, comparações, tempo em ms>


void countingSort(vector<int>& arr, int maxVal);


int main() {
    loginfo_t loginfo;
    vector<int> arr(100);
    

    auto start = std::chrono::steady_clock::now();
    auto finish = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds;

    for (int i = 0; i < 100; i++) {
        arr[i] = i + 1;
    }

    cout << "Array original: ";
    for (int i = 0; i < arr.size(); i++) {
        cout << arr[i] << " ";
    }
    cout << endl;


    start = std::chrono::steady_clock::now();
    countingSort(arr, 100);
    finish = std::chrono::steady_clock::now();
    elapsed_seconds = finish - start;
    get<2>(loginfo) = elapsed_seconds; 

    cout << "tempo = " << get<2>(loginfo).count() << "ms; " << endl; 

    cout << "Array ordenado: ";
    for (int i = 0; i < arr.size(); i++) {
        cout << arr[i] << " ";
    }
    cout << endl;

    return 0;
}

void countingSort(vector<int>& arr, int maxVal) {
    vector<int> count(maxVal + 1, 0);
    vector<int> output(arr.size(), 0);

    // conta ocorrencia de cada elemento
    for (int i = 0; i < arr.size(); i++) {
        count[arr[i]]++;
    }

    // calcula soma
    for (int i = 1; i < count.size(); i++) {
        count[i] += count[i-1];
    }

    // Coloca no output de forma ordenada
    for (int i = arr.size() - 1; i >= 0; i--) {
        output[count[arr[i]] - 1] = arr[i];
        count[arr[i]]--;
    }

    // Copia ordenado pro original
    for (int i = 0; i < arr.size(); i++) {
        arr[i] = output[i];
    }
}

